In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
filepath = lambda region: f"/home/sarth/rootdir/workdir/projects/Paper_Data_Latency/Revised_Figure03/v2_surr_{region}_test_metrics.nc"

# CAMELS-US

In [3]:
import warnings
warnings.filterwarnings('ignore')

import configparser
cfg = configparser.ConfigParser()
cfg.optionxform = str
cfg.read('/home/sarth/rootdir/datadir/assets/defaults.ini')
cfg = {s: dict(cfg.items(s)) for s in cfg.sections()}
PATHS = cfg['PATHS']

import os

#%% Region-Specific: CAMELS-US
DIRNAME = '03min_GloFAS_CAMELS-US'
SAVE_PATH = os.path.join(PATHS['devp_datasets'], DIRNAME)
resolution = 0.05
lon_360_180 = lambda x: (x + 180) % 360 - 180 # convert 0-360 to -180-180
lon_180_360 = lambda x: x % 360 # convert -180-180 to 0-360
region_bounds = {
    'minx': -130,
    'miny': 20,
    'maxx': -65,
    'maxy': 50
}
camels_attributes_graph = pd.read_csv(os.path.join(SAVE_PATH, 'graph_attributes.csv'), index_col=0)
camels_attributes_graph.index = camels_attributes_graph.index.map(lambda x: str(x).zfill(8))
camels_attributes_graph['huc_02'] = camels_attributes_graph['huc_02'].map(lambda x: str(x).zfill(2))
camels_graph = camels_attributes_graph.copy()
camels_graph = camels_graph[camels_graph['area_percent_difference'] < 10]
camels_graph = camels_graph[camels_graph['num_nodes'] > 1]
print(f"Number of CAMELS-US catmt's: {len(camels_graph)}")
del camels_attributes_graph

Number of CAMELS-US catmt's: 395


In [4]:
camels_graph

,huc_02,gauge_lon,gauge_lat,area_geospa_fabric,snapped_lon,snapped_lat,snapped_uparea,snapped_iou,area_percent_difference,num_nodes,num_edges
gauge_id,,,,,,,,,,,
06452000,10,-99.55649,43.74833,25817.78,-99.525,43.775,26081.820000,0.942874,1.022710,1158.0,1157.0
13340000,17,-116.25750,46.47833,14270.76,-116.275,46.475,14113.031000,0.940266,1.105257,657.0,656.0
06447000,10,-101.52487,43.75250,12869.46,-101.575,43.725,12933.430000,0.929852,0.497066,573.0,572.0
06360500,10,-100.84292,45.25582,12601.47,-100.875,45.275,12741.733000,0.926271,1.113074,584.0,583.0
06354000,10,-100.93444,46.37611,10626.74,-100.925,46.375,10708.444000,0.919406,0.768854,500.0,499.0
...,...,...,...,...,...,...,...,...,...,...,...
10310500,16,-119.79824,39.11324,47.65,-119.825,39.125,48.000664,0.621974,0.735912,2.0,1.0
01350140,02,-74.47293,42.42897,47.57,-74.525,42.425,45.708480,0.291629,3.913220,2.0,1.0
08271000,13,-105.53084,36.50836,46.39,-105.475,36.525,49.693077,0.571310,7.120237,2.0,1.0


In [5]:
prediction_huc = camels_graph['huc_02'].values.astype(int)
prediction_huc

array([10, 17, 10, 10, 10, 10, 10, 11, 10, 10,  3, 11, 11, 15, 10,  9, 11,
       10,  7,  7,  1, 12,  7, 17, 10, 10, 17, 17,  4,  5, 10, 17, 10,  3,
       17, 11,  9,  5,  7,  9,  1,  4, 11, 11, 12, 10, 11, 10,  8, 11, 18,
        4, 18, 18,  3, 11, 12, 17,  3,  5,  5, 12, 12,  1,  3,  2, 17, 10,
        7,  8, 15,  2, 18, 12,  2,  3,  4, 10, 10,  9, 10, 11, 11,  3, 10,
        3, 12, 10,  3,  3,  5,  8,  8,  3,  3, 15, 12,  4,  2,  2,  4, 12,
        7, 17,  6, 17,  7, 17,  3,  3,  2,  5,  4, 10, 10, 12, 10,  4, 10,
       11,  7, 12, 12,  5,  8, 10, 17,  7, 11,  7, 12,  1,  7,  7, 17, 12,
        2, 17,  2,  6, 17,  2, 12, 17, 12, 10, 18,  5, 11, 17,  5, 11,  3,
        5, 10,  3,  6,  7, 10, 15,  5, 11, 17, 10,  1, 10,  4, 18,  9, 10,
       10,  7,  2,  2, 17,  6, 17,  5,  7,  3,  3, 17, 17, 10,  8, 18, 12,
       18, 15, 17,  2,  4,  3,  3,  7, 18,  6, 17, 12, 17,  2,  7,  2,  5,
       18, 10,  3, 17,  5, 15, 11, 16,  7, 11,  3,  4,  3,  5,  3, 10, 10,
        2,  8,  2,  3,  3

In [6]:
ds = xr.open_dataset(filepath("camelsus"))
ds = ds[['NSE', 'F1_score']].to_dataframe().reset_index()
ds['gauge_order'] = ds['catmt_idx'] + 1
ds['lead_time'] += 1
ds['catmt_idx'] = prediction_huc[ds['catmt_idx'].values]
ds = ds.rename(columns={'catmt_idx': 'prediction_huc', 'lead_time': 'lead', 'F1_score': 'F1'})
ds = ds[['prediction_huc', 'gauge_order', 'lead', 'NSE', 'F1']]
ds

,prediction_huc,gauge_order,lead,NSE,F1
0,10,1,1,0.309689,0.438503
1,17,2,1,0.757776,0.731183
2,10,3,1,0.347020,0.558140
3,10,4,1,0.251076,0.445652
4,10,5,1,0.598807,0.429448
...,...,...,...,...,...
3945,16,391,10,0.650947,0.643678
3946,2,392,10,0.668706,0.853211
3947,13,393,10,-3.605224,0.359712
3948,17,394,10,0.934509,0.871287


In [7]:
save_path = '/home/sarth/rootdir/workdir/projects/Paper_Data_Latency/Revised_Statistical_Testing/Figure03'
ds.to_csv(f'{save_path}/camelsus.csv', index=False)

# CAMELS-IND

In [8]:
#%% Region-Specific: CAMELS-IND
DIRNAME = '03min_GloFAS_CAMELS-IND'
SAVE_PATH = os.path.join(PATHS['devp_datasets'], DIRNAME)
resolution = 0.05
lon_360_180 = lambda x: (x + 180) % 360 - 180 # convert 0-360 to -180-180
lon_180_360 = lambda x: x % 360 # convert -180-180 to 0-360
region_bounds = {
    'minx': 66,
    'miny': 5,
    'maxx': 100,
    'maxy': 30
}
camels_attributes_graph = pd.read_csv(os.path.join(SAVE_PATH, 'graph_attributes.csv'), index_col=0)
camels_attributes_graph.index = camels_attributes_graph.index.map(lambda x: str(x).zfill(5))
camels_attributes_graph['huc_02'] = camels_attributes_graph['huc_02'].map(lambda x: str(x).zfill(2))
camels_graph = camels_attributes_graph.copy()
camels_graph = camels_graph[camels_graph['ghi_area'] <= 30000]
camels_graph = camels_graph[camels_graph['area_percent_difference'] < 10]
camels_graph = camels_graph[camels_graph['num_nodes'] > 1]
camels_graph = camels_graph.rename(columns = {'ghi_lon': 'gauge_lon', 'ghi_lat': 'gauge_lat'})
print(f"Number of catmt's: {len(camels_graph)}")
del camels_attributes_graph

Number of catmt's: 191


In [9]:
camels_graph

,huc_02,gauge_lon,gauge_lat,ghi_area,cwc_lon,cwc_lat,cwc_area,cwc_site_name,ghi_stn_id,river_basin,cwc_river,flow_availability,snapped_lon,snapped_lat,snapped_uparea,snapped_iou,area_percent_difference,num_nodes,num_edges
gauge_id,,,,,,,,,,,,,,,,,,,
14015,14,73.11033,18.73540,125.7,73.1108,18.7367,125.0,Pen,wfrn_penxx,WFRN,Bhogeswari,31.36,73.125,18.725,116.744995,0.650870,7.124106,4.0,3.0
15006,15,74.88124,13.51876,299.6,74.8800,13.5214,253.0,Avershe,wfrs_avers,WFRS,Seetha,39.00,74.925,13.475,329.444850,0.607418,9.961565,11.0,10.0
05025,05,78.05617,11.93959,356.0,78.0572,11.9383,362.0,Thoppur,cauv_thopp,Cauvery,Cauvery/Thoppaiyar,31.52,78.125,11.975,331.356480,0.634196,6.922338,11.0,10.0
15032,15,74.98123,13.29791,356.9,74.9806,13.2942,327.0,Yennehole,wfrs_yenne,WFRS,Swarna,70.67,74.975,13.275,329.717100,0.755086,7.616389,11.0,10.0
15007,15,76.84792,8.71458,555.8,76.8500,8.7150,540.0,Ayilam,wfrs_ayila,WFRS,Vamanapuram,93.18,76.875,8.725,578.080800,0.773097,4.008784,19.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
04038,04,75.13955,17.97291,22326.1,75.1392,17.9722,22856.0,Narsingpur,kris_narsi,Krishna,Krishna/Bhima,95.30,75.125,17.975,22149.584000,0.953356,0.790624,758.0,757.0
04057,04,76.96458,15.66043,23527.1,76.9628,15.6611,23500.0,T. Ramapuram (Seasonal),kris_trama,Krishna,Krishna/Tungabhadra/Hagari,84.89,76.975,15.625,23463.662000,0.941489,0.269636,786.0,785.0
03057,03,80.68958,19.29779,24041.7,80.6892,19.2989,24210.0,Mirdapalli,goda_mirda,Godavari,Godavari/Pranhita/Indravati,56.17,80.675,19.275,23877.793000,0.940053,0.681758,821.0,820.0


In [10]:
prediction_huc = camels_graph['huc_02'].values.astype(int)
prediction_huc

array([14, 15,  5, 15, 15,  4, 15, 14, 15, 17, 15, 15, 15, 14, 12, 15, 15,
       14, 15,  7, 14,  5,  8,  3, 15,  3,  5, 15,  5, 15,  5, 17,  3,  5,
        3, 15,  4, 12,  5, 14,  5, 15,  3, 12,  3,  4,  5, 14,  3, 12,  3,
        7,  3, 11,  3,  4, 13,  3,  3,  4, 15, 14,  4,  3, 16,  4,  3,  8,
       12,  4, 12, 17,  4,  4,  9,  8,  4,  4,  3, 10, 12,  4, 15,  8, 11,
        3,  4,  3, 17,  7,  8,  4, 15,  5, 11,  4,  8,  5,  3, 10, 13,  3,
       10, 17, 12, 12, 12, 12, 15,  4,  6,  3,  8, 17,  8, 12,  3,  3, 17,
        5,  4,  3,  4,  3,  3,  5, 17, 15,  8,  9,  6, 11, 13,  4, 16, 16,
        5,  8,  9, 17,  3,  4, 16,  5, 13,  4,  9,  3,  8, 10,  7,  5, 13,
       16,  8,  4,  3,  3,  7,  4,  3,  4, 17,  8,  4,  4,  7,  9,  6,  3,
       17,  6,  4,  4,  3, 13, 17, 10,  8,  3, 11,  3,  3,  3,  8,  7,  4,
        4,  3, 12,  8])

In [11]:
ds = xr.open_dataset(filepath("camelsind"))
ds = ds[['NSE', 'F1_score']].to_dataframe().reset_index()
ds['gauge_order'] = ds['catmt_idx'] + 1
ds['lead_time'] += 1
ds['catmt_idx'] = prediction_huc[ds['catmt_idx'].values]
ds = ds.rename(columns={'catmt_idx': 'prediction_huc', 'lead_time': 'lead', 'F1_score': 'F1'})
ds = ds[['prediction_huc', 'gauge_order', 'lead', 'NSE', 'F1']]
ds

,prediction_huc,gauge_order,lead,NSE,F1
0,14,1,1,0.503827,0.687500
1,15,2,1,0.363504,0.644295
2,5,3,1,0.418737,0.583333
3,15,4,1,0.331176,0.662162
4,15,5,1,0.554633,0.760000
...,...,...,...,...,...
1905,4,187,10,0.613353,0.597222
1906,4,188,10,0.271453,0.562874
1907,3,189,10,0.474276,0.819876
1908,12,190,10,0.485583,0.614173


In [13]:
save_path = '/home/sarth/rootdir/workdir/projects/Paper_Data_Latency/Revised_Statistical_Testing/Figure03'
ds.to_csv(f'{save_path}/camelsind.csv', index=False)

# HYSETS

In [ ]:
#%% Region-Specific: hysets
DIRNAME = '03min_GloFAS_hysets'
SAVE_PATH = os.path.join(PATHS['devp_datasets'], DIRNAME)
resolution = 0.05
lon_360_180 = lambda x: (x + 180) % 360 - 180 # convert 0-360 to -180-180
lon_180_360 = lambda x: x % 360 # convert -180-180 to 0-360
region_bounds = {
    'minx': -130,
    'miny': 20,
    'maxx': -65,
    'maxy': 50
}

catmt_graph = pd.read_csv(os.path.join(SAVE_PATH, 'nested_gauges', 'graph_attributes_with_nesting.csv'))
# catmt_graph.index = catmt_graph.index.map(lambda x: str(x).zfill(8))
catmt_graph['huc_02'] = catmt_graph['huc_02'].map(lambda x: str(x).zfill(2))
# catmt_graph = catmt_graph[catmt_graph['area_percent_difference'] < 10]
# catmt_graph = catmt_graph[catmt_graph['num_nodes'] > 1]
print(f"Number of catmt's: {len(catmt_graph)}")

# catmt_graph = catmt_graph[catmt_graph['nesting'].isin(['nested_downstream'])]
# catmt_graph = catmt_graph.reset_index()
print(f"Number of catmt's: {len(catmt_graph)}")

Number of catmt's: 5149
Number of catmt's: 5149


In [17]:
DIRNAME = '03min_GloFAS_hysets'
AREA_SOURCE = 'gauge_uparea'
# GAUGE_SOURCE = 'IndiaWRIS'
SAVE_PATH = os.path.join(PATHS['devp_datasets'], DIRNAME)
SCALING_PATH = os.path.join(PATHS['devp_datasets'], "03min_GloFAS_CAMELS-US")
resolution = 0.05
lon_360_180 = lambda x: (x + 180) % 360 - 180 # convert 0-360 to -180-180
lon_180_360 = lambda x: x % 360 # convert -180-180 to 0-360
region_bounds = {
    'minx': -130,
    'miny': 20,
    'maxx': -65,
    'maxy': 50
}
catmt_graph = pd.read_csv(os.path.join(SAVE_PATH, 'nested_gauges', 'catmts.csv'))
catmt_graph['huc_02'] = catmt_graph['huc_02'].map(lambda x: str(x).zfill(2))
# catmt_graph = catmt_graph[catmt_graph['nesting'].isin(['not_nested', 'nested_downstream'])]
print(f"Number of catmt's with nesting: {len(catmt_graph)}")
catmt_graph

Number of catmt's with nesting: 5149


,huc_02,gauge_lon,gauge_lat,gauge_uparea,snapped_lon,snapped_lat,uparea_snapped,iou_snapped,area_percent_difference,num_nodes,num_edges,gauge_id,nesting
0,01,-71.164783,42.948976,41.438557,-71.175,42.925,45.347492,0.539036,9.433087,2.0,1.0,hysets_010735562,not_nested
1,01,-72.796765,41.295930,43.553455,-72.775,41.325,46.490100,0.355355,6.742623,2.0,1.0,hysets_01195339,not_nested
2,01,-71.956462,41.765376,43.926884,-71.975,41.825,46.136967,0.416741,5.031277,2.0,1.0,hysets_01126600,not_nested
3,01,-71.111111,42.561111,45.008418,-71.125,42.525,45.636566,0.333445,1.395621,2.0,1.0,hysets_01101344,not_nested
4,01,-72.202222,41.368611,45.528262,-72.225,41.425,46.402153,0.568377,1.919445,2.0,1.0,hysets_011277905,not_nested
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5144,18,-122.186665,40.288488,25356.089877,-122.175,40.325,26037.320000,0.784760,2.686654,1116.0,1115.0,hysets_11377100,nested_upstream
5145,18,-122.223054,40.264044,25376.472968,-122.225,40.275,26060.930000,0.784653,2.697211,1117.0,1116.0,hysets_11377200,nested_upstream
5146,18,-121.994142,39.457663,33606.175081,-121.975,39.575,31627.365000,0.770517,5.888235,1352.0,1351.0,hysets_11389000,nested_upstream
5147,18,-122.000251,39.214057,33971.062974,-122.025,39.275,31842.406000,0.764659,6.266087,1361.0,1360.0,hysets_11389500,nested_upstream


In [18]:
prediction_huc = catmt_graph['huc_02'].values.astype(int)
prediction_huc

array([ 1,  1,  1, ..., 18, 18, 18], shape=(5149,))

In [19]:
ds = xr.open_dataset(filepath("hysets"))
ds = ds[['NSE', 'F1_score']].to_dataframe().reset_index()
ds['gauge_order'] = ds['catmt_idx'] + 1
ds['lead_time'] += 1
ds['catmt_idx'] = prediction_huc[ds['catmt_idx'].values]
ds = ds.rename(columns={'catmt_idx': 'prediction_huc', 'lead_time': 'lead', 'F1_score': 'F1'})
ds = ds[['prediction_huc', 'gauge_order', 'lead', 'NSE', 'F1']]
ds

,prediction_huc,gauge_order,lead,NSE,F1
0,1,1,1,0.661096,0.750000
1,1,2,1,0.010831,0.797342
2,1,3,1,0.743657,0.823045
3,1,4,1,0.597180,0.757962
4,1,5,1,-2.061036,0.694915
...,...,...,...,...,...
51485,18,5145,10,-3.191533,0.571429
51486,18,5146,10,-3.339760,0.532468
51487,18,5147,10,-1.651893,0.658065
51488,18,5148,10,-2.721652,0.589744


In [20]:
save_path = '/home/sarth/rootdir/workdir/projects/Paper_Data_Latency/Revised_Statistical_Testing/Figure03'
ds.to_csv(f'{save_path}/hysets.csv', index=False)